In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/587.7 MB ? eta -:--:--ERROR: Operation cancelled by user

Aborted.


In [ ]:
import spacy
import pandas as pd
import re
import numpy as np

In [ ]:
import pandas as pd
data = pd.read_csv("/content/data_en.csv")
data.head()

In [ ]:
data.text = data.text.apply(lambda x: x.lower())
data.head()

,text,sentiment
0,a heretical book that utters many blasphemies....,0
1,started out ok... turned to crap shrortly ther...,0
2,no brainer set-up: i rent a cottage on a farm ...,1
3,"if you could only buy 1 book for your child, t...",1
4,digitech bp80 bass multieffects processor with...,1


In [ ]:
data.shape

(100000, 2)

In [ ]:
data.sentiment.value_counts()

,count
sentiment,
0,50147
1,49853


In [ ]:
data.isnull().sum()

,0
text,0
sentiment,0


In [ ]:
data.duplicated().sum()

0

In [ ]:
#data.sentiment.replace({"Positive":1,"Negative":0},inplace=True)
#data.head()

In [ ]:
nlp = spacy.load('en_core_web_lg')

def text_preprocessing(text):
  lower = text.lower()
  doc = nlp(lower)
  tokens = []
  for token in doc:
    if token.pos_ not in ["SPACE","PUNCT","X"]:
      tokens.append(token.lemma_)
  return " ".join(tokens)



In [ ]:
data.head()

,text,sentiment
0,A heretical book that utters many blasphemies....,0
1,Started out ok... turned to crap shrortly ther...,0
2,No brainer set-up: I rent a cottage on a farm ...,1
3,"If you could only buy 1 book for your child, t...",1
4,Digitech BP80 Bass multieffects processor with...,1


In [ ]:
data.text = data.text.apply(text_preprocessing)
data.head()

In [ ]:
data.to_csv("/content/drive/MyDrive/data_en_preprocessed.csv",index=False)

In [ ]:
data = pd.read_csv("/content/data_en_preprocessed.csv")
data.head()

,text,sentiment
0,a heretical book that utter many blasphemy boy...,0
1,start out ok turn to crap shrortly thereafter ...,0
2,no brainer set up I rent a cottage on a farm a...,1
3,if you could only buy 1 book for your child th...,1
4,digitech bp80 bass multieffects processor with...,1


In [ ]:
X = data.text
y = data.sentiment

In [ ]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(data, test_size=0.2, stratify=y, random_state=42)

In [ ]:
train_data.text[0]

'a heretical book that utter many blasphemy boyd hit a new low after call the omnipotent god of historic christianity a dead corpse he now try to win his father to finite godism'

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
X_train[0]

'a heretical book that utter many blasphemy boyd hit a new low after call the omnipotent god of historic christianity a dead corpse he now try to win his father to finite godism'

In [ ]:
X_train.shape

(80000,)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
# Création du pipeline
pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Étape de vectorisation du texte
    ('classifier', LogisticRegression())  # Modèle de classification
])

# Entraînement du pipeline
pipeline.fit(X_train, y_train)


Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [ ]:
from sklearn.metrics import accuracy_score
# Prédiction
y_pred = pipeline.predict(X_test)

# Évaluation
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy

0.8905

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89     10029
           1       0.89      0.88      0.89      9971

    accuracy                           0.89     20000
   macro avg       0.89      0.89      0.89     20000
weighted avg       0.89      0.89      0.89     20000



In [ ]:
from xgboost import XGBClassifier
pip_line = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Étape de vectorisation du texte
    ('classifier', XGBClassifier(n_estimators=200))  # Modèle de classification
])

# Entraînement du pipeline
pip_line.fit(X_train, y_train)

In [ ]:
y_pred = pip_line.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.87      0.88     10041
           1       0.87      0.88      0.88      9959

    accuracy                           0.88     20000
   macro avg       0.88      0.88      0.88     20000
weighted avg       0.88      0.88      0.88     20000



In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipline = Pipeline([
    ('vectorizer', TfidfVectorizer()),  # Étape de vectorisation du texte
    ('classifier', RandomForestClassifier())  # Modèle de classification
])

# Entraînement du pipeline
pipline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [ ]:
y_pred = pipline.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86     10041
           1       0.88      0.83      0.85      9959

    accuracy                           0.86     20000
   macro avg       0.86      0.86      0.86     20000
weighted avg       0.86      0.86      0.86     20000



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:

# 1. Tokenize the text data
tokenizer = Tokenizer(num_words=100000)  # Limit vocabulary size
tokenizer.fit_on_texts(data["text"])
sequences = tokenizer.texts_to_sequences(data["text"])

# 2. Pad sequences to ensure uniform length
X = pad_sequences(sequences, maxlen=100)  # Adjust maxlen as needed

# 3. Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X

array([[    0,     0,     0, ...,     5, 17839, 45444],
       [    6,    13,     1, ...,     4,  4347,    49],
       [   13,    58,  1250, ...,     3,    96,  8379],
       ...,
       [    0,     0,     0, ...,  3024,    25,   272],
       [    0,     0,     0, ...,   741,   982,  3032],
       [  572,    66,     3, ...,  8352,   261,     6]], dtype=int32)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

# Création du modèle LSTM avec Dropout
lstm = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=100),  # Couche d'embedding
    Bidirectional(LSTM(64, return_sequences=True)),  # LSTM bidirectionnelle avec retour des séquences
    Dropout(0.5),  # Dropout de 50% après la première couche LSTM
    Bidirectional(LSTM(64)),  # Deuxième couche LSTM bidirectionnelle
    Dropout(0.5),  # Dropout de 50% après la deuxième couche LSTM
    Dense(64, activation="relu"),  # Couche dense
    Dropout(0.5),  # Dropout de 50% avant la couche de sortie
    Dense(1, activation='sigmoid')  # Sortie avec activation sigmoid pour classification binaire
])

# Compilation du modèle
lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
lstm.fit(X_train,y_train,epochs=10, batch_size=100 ,validation_data=(X_test,y_test))

Epoch 1/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 28s 23ms/step - accuracy: 0.7384 - loss: 0.4783 - val_accuracy: 0.8799 - val_loss: 0.2864
Epoch 2/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 36s 23ms/step - accuracy: 0.9052 - loss: 0.2445 - val_accuracy: 0.8917 - val_loss: 0.2675
Epoch 3/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.9279 - loss: 0.1945 - val_accuracy: 0.8993 - val_loss: 0.2752
Epoch 4/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 22s 25ms/step - accuracy: 0.9434 - loss: 0.1573 - val_accuracy: 0.8967 - val_loss: 0.2757
Epoch 5/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 19s 22ms/step - accuracy: 0.9557 - loss: 0.1274 - val_accuracy: 0.8902 - val_loss: 0.3168
Epoch 6/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 22s 24ms/step - accuracy: 0.9645 - loss: 0.1068 - val_accuracy: 0.8883 - val_loss: 0.3733
Epoch 7/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.9706 - loss: 0.0928 - val_accuracy: 0.8893 - val_loss: 0.3707
Epoch 8/10
800/800 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.9784 - loss: 0.0716 - 

In [ ]:
lstm.evaluate(X_test,y_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8870 - loss: 0.4522


[0.4689784049987793, 0.8838000297546387]

In [ ]:
from sklearn.metrics import classification_report
y_pred = lstm.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_test,y_pred))

625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step
              precision    recall  f1-score   support

           0       0.88      0.89      0.89     10041
           1       0.89      0.88      0.88      9959

    accuracy                           0.88     20000
   macro avg       0.88      0.88      0.88     20000
weighted avg       0.88      0.88      0.88     20000



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Exemple de phrase à tester
phrase = "I'm glad to see you!"

# Tokenisation de la phrase
sequence = tokenizer.texts_to_sequences([phrase])

# Padding de la séquence pour qu'elle ait une longueur de 100 (comme défini dans le modèle)
padded_sequence = pad_sequences(sequence, maxlen=100)

# Faire une prédiction avec le modèle LSTM
prediction = lstm.predict(padded_sequence)

# Afficher le résultat
print(f"Prédiction : {prediction}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
Prédiction : [[0.824936]]


In [ ]:
gru = Sequential([
    Embedding(input_dim=10000, output_dim=16, input_length=100),
    Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    Bidirectional(tf.keras.layers.GRU(64)),
    Dense(64, activation="relu"),  # Couche dense
    Dense(1, activation='sigmoid')
])
gru.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
gru.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_8 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_9 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
gru.evaluate(X_test,y_test)

In [ ]:
y_pred = gru.predict(X_test)
y_pred = np.where(y_pred > 0.5, 1, 0)
print(classification_report(y_test,y_pred))

In [ ]:
df = data.copy()
df.head()

,text,sentiment
0,a heretical book that utter many blasphemy boy...,0
1,start out ok turn to crap shrortly thereafter ...,0
2,no brainer set up I rent a cottage on a farm a...,1
3,if you could only buy 1 book for your child th...,1
4,digitech bp80 bass multieffects processor with...,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       100000 non-null  object
 1   sentiment  100000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


In [ ]:
df["label"] = "__label__" + df["sentiment"].astype(str)
df.head()

,text,sentiment,label
0,a heretical book that utter many blasphemy boy...,0,__label__0
1,start out ok turn to crap shrortly thereafter ...,0,__label__0
2,no brainer set up I rent a cottage on a farm a...,1,__label__1
3,if you could only buy 1 book for your child th...,1,__label__1
4,digitech bp80 bass multieffects processor with...,1,__label__1


In [ ]:
df["data"] = df["label"] + " " + df["text"]
df.head()

,text,sentiment,label,data
0,a heretical book that utter many blasphemy boy...,0,__label__0,__label__0 a heretical book that utter many bl...
1,start out ok turn to crap shrortly thereafter ...,0,__label__0,__label__0 start out ok turn to crap shrortly ...
2,no brainer set up I rent a cottage on a farm a...,1,__label__1,__label__1 no brainer set up I rent a cottage ...
3,if you could only buy 1 book for your child th...,1,__label__1,__label__1 if you could only buy 1 book for yo...
4,digitech bp80 bass multieffects processor with...,1,__label__1,__label__1 digitech bp80 bass multieffects pro...


In [ ]:
df.drop(["sentiment","label","text"],axis=1,inplace=True)
df.head()

,data
0,__label__0 a heretical book that utter many bl...
1,__label__0 start out ok turn to crap shrortly ...
2,__label__1 no brainer set up I rent a cottage ...
3,__label__1 if you could only buy 1 book for yo...
4,__label__1 digitech bp80 bass multieffects pro...


In [ ]:
train_fasttext_en, test_fasttext_en = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_fasttext_en.to_csv("fasttext_en_train.train",index=False,header=False)
test_fasttext_en.to_csv("fasttext_en_test.test",index=False,header=False)

In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296187 sha256=918b8bba4d6f8cff51adbec3a5e84e8d1d840db264da378b2b27c68007d05a90
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import fasttext

model = fasttext.train_supervised(input="/content/fasttext_en_train.train")

In [ ]:
model.test("/content/fasttext_en_test.test")

(19583, 0.8570699075728948, 0.8570699075728948)

In [ ]:
model.predict("I love you")

(('__label__1',), array([1.00001001]))

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

In [ ]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer.encode_plus(text, truncation=True,
                                          padding='max_length',
                                          max_length=self.max_len
                                          )
    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [ ]:
X = data["text"].tolist()
y = data["sentiment"].tolist()

dataset = CustomDataset(X, y, tokenizer)

In [ ]:
dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(exemple):
    labels = exemple.label_ids
    preds = exemple.predictions.argmax(-1)

    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average='weighted')

    return {'accuracy': accuracy, 'f1': f1}

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 16
model_name = "distilbert-base-uncased"

args = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    num_train_epochs=1,
    evaluation_strategy="epoch"
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.159400,0.165812,0.946000,0.946000


TrainOutput(global_step=5000, training_loss=0.19724760437011718, metrics={'train_runtime': 4128.5014, 'train_samples_per_second': 19.377, 'train_steps_per_second': 1.211, 'total_flos': 1.059739189248e+16, 'train_loss': 0.19724760437011718, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.16581152379512787,
 'eval_accuracy': 0.946,
 'eval_f1': 0.9460002257208149,
 'eval_runtime': 349.2341,
 'eval_samples_per_second': 57.268,
 'eval_steps_per_second': 3.579,
 'epoch': 1.0}

In [ ]:
trainer.save_model(model_name)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
